<a href="https://colab.research.google.com/github/olonok69/LLM_Notebooks/blob/main/langchain/langgraph/Langgraph_multi_modal_Agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Langgraph multi modal Agent

### Tools
Tools are external resources, services, or APIs that an LLM agent can access and utilize to expand its capabilities and perform specific tasks. These supplementary components allow the agent to go beyond its core language processing abilities, enabling it to interact with external systems, retrieve information, or execute actions that would otherwise be outside its scope. By integrating tools, LLM agents can provide more comprehensive and practical solutions to user queries and commands.

A tool consists of:

- The name of the tool.
- A description of what the tool does.
- A JSON schema defining the inputs to the tool.
- A function (and, optionally, an async variant of the function)

When a tool is bound to a model, the name, description and JSON schema are provided as context to the model. Given a list of tools and a set of instructions, a model can request to call one or more tools with specific inputs.

https://python.langchain.com/v0.2/docs/concepts/#tools


In [ ]:
%%capture --no-stderr
%pip install --quiet -U langgraph langchain_anthropic langchain_openai langchain-google-genai langchain-community  langchain-chroma  pandas  ipywidgets pillow

In [ ]:
import os
from google.colab import userdata

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = userdata.get('LANGCHAIN_API_KEY')
os.environ["LANGCHAIN_PROJECT"] = "lg-multimodal-agent"
os.environ["OPENAI_API_KEY"] =  userdata.get("KEY_OPENAI")
os.environ["ANTHROPIC_API_KEY"]=  userdata.get("ANTROPIC_KEY")
GEMINI_API_KEY = userdata.get("GEMINI_API_KEY")

In [ ]:
import pandas as pd
from collections import Counter
from langchain_core.tools import tool


def read_travel_data(file_path: str = "/content/drive/MyDrive/data/synthetic_travel_data.csv") -> pd.DataFrame:
    """Read travel data from CSV file"""
    try:
        df = pd.read_csv(file_path)
        return df
    except FileNotFoundError:
        return pd.DataFrame(
            columns=[
                "Name",
                "Current_Location",
                "Age",
                "Past_Travel_Destinations",
                "Number_of_Trips",
                "Flight_Number",
                "Departure_City",
                "Arrival_City",
                "Flight_Date",
            ]
        )


@tool
def compare_and_recommend_destination(name: str) -> str:
    """This tool is used to check which destinations user has already traveled.
    Use name of the user to fetch the information about that user.
    If user has already been to a city then do not recommend that city.

    Args:
        name (str): Name of the user.
    Returns:
        str: Destination to be recommended.

    """

    df = read_travel_data()

    if name not in df["Name"].values:
        return "User not found in the travel database."

    user_data = df[df["Name"] == name].iloc[0]
    current_location = user_data["Current_Location"]
    age = user_data["Age"]
    past_destinations = user_data["Past_Travel_Destinations"].split(", ")

    # Get all past destinations of users with similar age (±5 years) and same current location
    similar_users = df[
        (df["Current_Location"] == current_location)
        & (df["Age"].between(age - 5, age + 5))
    ]
    all_destinations = [
        dest
        for user_dests in similar_users["Past_Travel_Destinations"].str.split(", ")
        for dest in user_dests
    ]

    # Count occurrences of each destination
    destination_counts = Counter(all_destinations)

    # Remove user's current location and past destinations from recommendations
    for dest in [current_location] + past_destinations:
        if dest in destination_counts:
            del destination_counts[dest]

    if not destination_counts:
        return f"No new recommendations found for users in {current_location} with similar age."

    # Get the most common destination
    recommended_destination = destination_counts.most_common(1)[0][0]

    return f"Based on your current location ({current_location}), age ({age}), and past travel data, we recommend visiting {recommended_destination}."

In [ ]:
df = read_travel_data()
df

In [ ]:
df[df.Name=="Christian Morales"]

In [ ]:
df[df['Current_Location']== "Venice"]

In [ ]:
compare_and_recommend_destination("Kevin Garcia")

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
# Set up the model
from langchain_anthropic import ChatAnthropic
from langchain_openai import ChatOpenAI

llm = ChatAnthropic(model="claude-3-5-sonnet-20240620")
llm = ChatOpenAI(model="gpt-4o")

In [ ]:
tools = [compare_and_recommend_destination]

In [ ]:
llm_with_tools = llm.bind_tools(tools)

In [ ]:
import operator
from typing import Annotated, Sequence, TypedDict

from langchain_core.messages import BaseMessage, HumanMessage
from langchain_core.messages import ToolMessage
from langgraph.prebuilt import ToolInvocation

class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], operator.add]



In [ ]:
from typing import Annotated

from typing_extensions import TypedDict

from langgraph.graph import StateGraph, START, MessagesState, END
from langgraph.graph.message import add_messages


class State(TypedDict):
    messages: Annotated[list, add_messages]


graph_builder = StateGraph(State)


def chatbot(state: State):
    """Always use tools to fulfill user requests.
    1. If you do not have enough inputs to execute a tool then you can ask for more information.
    2. If user has uploaded image and 'image_processing_node' has returned city and activity then use that information to call 'chatbot'
    """
    # Filter out messages with image type
    # text_messages = [msg for msg in state["messages"] if msg['content'][0].get("type") != "image"]
    text_messages = [
        msg for msg in state["messages"]
        if not (isinstance(msg.content, list) and msg.content[0].get("type") == "image_url")
    ]

    # Invoke LLM with only text messages
    return {"messages": [llm_with_tools.invoke(text_messages)]}


graph_builder.add_node("chatbot", chatbot)

In [ ]:
from langgraph.prebuilt import ToolNode, tools_condition

tool_node = ToolNode(tools)
graph_builder.add_node("tools", tool_node)

graph_builder.add_conditional_edges(
    "chatbot",
    tools_condition,
    {"tools": "tools", "__end__": "__end__"},
)

In [ ]:
def process_image_input(state):
    """
    Process image input. This tool will return activity shown in this image.
    """
    last_message = state["messages"][-1].content[0]
    input_image = last_message['image_url']['url']
    print(input_image)
    message = HumanMessage(
        content=[
            {"type": "text", "text": "Which activity is shown in this image?"},
            {
                "type": "image_url",
                # "image_url": {"url": {input_image}},
                "image_url": {"url": f"data:image/jpeg;base64,{input_image}"},
            },
        ],
    )
    response = llm.invoke([message])
    print(response.content)
    output = {
        "messages": [
            HumanMessage(
                content=f"Image information: {response.content}", name="image_description"
            )
        ],
    }
    return output

In [ ]:
graph_builder.add_node("image_processing_node", process_image_input)

In [ ]:
def is_image_node(state):
    messages = state["messages"]
    last_message = messages[-1]
    if hasattr(last_message, "content") and isinstance(last_message.content, list):
        for item in last_message.content:
            if isinstance(item, dict) and item.get("type") == "image_url":
                return True
    return False

In [ ]:
from langgraph.checkpoint.memory import MemorySaver
from IPython.display import Image, display

memory = MemorySaver()

# Any time a tool is called, we return to the chatbot to decide the next step
graph_builder.add_edge("tools", "chatbot")
graph_builder.add_edge("image_processing_node", "chatbot")
graph_builder.add_conditional_edges(START, is_image_node, {True: "image_processing_node", False:"chatbot"})

graph = graph_builder.compile(checkpointer=memory)

try:
    display(Image(graph.get_graph().draw_mermaid_png()))
except Exception:
    # This requires some extra dependencies and is optional
    pass


In [ ]:
from IPython.display import display, clear_output
import ipywidgets as widgets

from PIL import Image
import io
import base64




In [ ]:
img = Image.open("/content/drive/MyDrive/data/images/images/zermatt.jpg")
img.resize((256, 256))

In [ ]:
buffered = io.BytesIO()
img.save(buffered, format=img.format)
img_str = base64.b64encode(buffered.getvalue()).decode("utf-8")

In [ ]:
from langchain_core.messages import HumanMessage
import pprint
config = {"configurable": {"thread_id": "20"}}
message = HumanMessage(
    content=[
        {
            "type": "image_url",
            "image_url": {"url": img_str},
        },
    ],
)
pprint.pprint(graph.invoke({"messages": message}, config)["messages"][-1].content)

In [ ]:
pprint.pprint(
    graph.invoke(
        {
            "messages": [
                (
                    "user",
                    "My name is Kevin Garcia",
                )
            ]
        },
        config,
    )["messages"][-1].content
)

In [ ]:
pprint.pprint(
    graph.invoke(
        {
            "messages": [
                (
                    "user",
                    "yes I want some suggestions based on the picture",
                )
            ]
        },
        config,
    )["messages"][-1].content
)

In [ ]:
pprint.pprint(
    graph.invoke(
        {
            "messages": [
                (
                    "user",
                    "yes I want only destinations in Europe ",
                )
            ]
        },
        config,
    )["messages"][-1].content
)